In [ ]:
#Code returning the properties of networkx graph

def list_properties_nx(G):
    node_properties = set()
    edge_properties = set()
    
    # Get node properties
    for _, node_data in G.nodes(data=True):
        node_properties.update(node_data.keys())
    
    # Get edge properties
    for _, _, edge_data in G.edges(data=True):
        edge_properties.update(edge_data.keys())
    
    return {
        'node_properties': node_properties,
        'edge_properties': edge_properties
    }

# Use the function to list properties for 'AT_2008'
properties_AT_2008 = list_properties_nx(datasets_nx['AT_2008'])
properties_AT_2008


In [ ]:
#Code to create projections of the bipartite graph in networkx
# Use the 'label' property to distinguish between issuers and winners
issuers_set = {n for n, d in datasets_nx['AT_2008'].nodes(data=True) if d['label'].endswith('_i')}
winners_set = {n for n, d in datasets_nx['AT_2008'].nodes(data=True) if d['label'].endswith('_w')}

# Check if the graph is bipartite
is_bipartite = bipartite.is_bipartite_node_set(datasets_nx['AT_2008'], issuers_set)

# Now we project the graph onto issuers and winners
issuer_projection = bipartite.projected_graph(datasets_nx['AT_2008'], issuers_set)
winner_projection = bipartite.projected_graph(datasets_nx['AT_2008'], winners_set)

# Get basic information about the projected graphs
num_nodes_issuers = issuer_projection.number_of_nodes()
num_edges_issuers = issuer_projection.number_of_edges()
num_nodes_winners = winner_projection.number_of_nodes()
num_edges_winners = winner_projection.number_of_edges()

is_bipartite, num_nodes_issuers, num_edges_issuers, num_nodes_winners, num_edges_winners



In [ ]:
## Code to convert projections from networkx to graph-tool

from graph_tool.all import Graph

def convert_networkx_to_graphtool(G_nx):
    # Create a new undirected graph-tool Graph
    G_gt = Graph(directed=False)
    
    # Add node properties with specific types
    pos_property = G_gt.new_vertex_property("vector<double>")
    label_property = G_gt.new_vertex_property("string")
    
    # Create a mapping from networkx nodes to graph-tool vertices
    vertex_map = {}
    for node, attr in G_nx.nodes(data=True):
        v = G_gt.add_vertex()
        vertex_map[node] = v
        pos_property[v] = attr.get('_pos', [0.0, 0.0])  # Provide a default value if '_pos' is not present
        label_property[v] = attr.get('label', '')  # Provide a default empty label if 'label' is not present
    
    # Add edges to graph-tool Graph (without properties, since they're not present)
    for u, v in G_nx.edges():
        G_gt.add_edge(vertex_map[u], vertex_map[v])
    
    # Set graph properties
    G_gt.graph_properties["name"] = G_gt.new_graph_property("string", "eu_procurements_alt (AT_2008)")
    G_gt.graph_properties["description"] = G_gt.new_graph_property("string", "These 234 networks represent the annual national public "
                                                                        "procurement markets of 26 European countries from "
                                                                        "2008-2016, inclusive. Data is sourced from Tenders Electronic "
                                                                        "Daily (TED), the official procurement portal of the "
                                                                        "European Union...")
    
    return G_gt, pos_property, label_property

# Convert networkx projections to graph-tool graphs
issuer_projection_gt, pos_issuer, label_issuer = convert_networkx_to_graphtool(issuer_projection)
winner_projection_gt, pos_winner, label_winner = convert_networkx_to_graphtool(winner_projection)
